<a href="https://colab.research.google.com/github/M2kishore/Covid_Detection_using_CT_scans_ML/blob/main/ML_feature_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [68]:
%matplotlib inline
from glob import glob
import cv2
import PIL
import tqdm
import pickle
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from torchvision import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, accuracy_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from skimage.filters import gabor
from skimage.feature import local_binary_pattern, greycomatrix, greycoprops

## Decomposition imports

In [126]:
from sklearn.decomposition import PCA,FastICA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.manifold import LocallyLinearEmbedding
from sklearn.manifold import TSNE

## Ensemble

In [69]:
from sklearn.ensemble import RandomForestClassifier

## Evaluation Metrics

In [77]:
from sklearn.metrics import classification_report,confusion_matrix

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/Amrita/Semester 6/semester_projects/Machine Learning/Dataset/

/content/drive/MyDrive/Amrita/Semester 6/semester_projects/Machine Learning/Dataset


In [ ]:
%ls

COVID/              covidLable.pckl        non-COVID/
covidFeatures.pckl  feature_extracted.csv  output_convo.jpg


In [ ]:
#define size to which images are to be resized
IMAGE_SIZE = [200, 200] # feel free to change depending on dataset

# training config:
epochs = 500
batch_size = 32

#define paths
covid_path = 'COVID'
noncovid_path = 'non-COVID'

# Use glob to grab images from path .jpg or jpeg
covid_files = glob(covid_path + '/*')
noncovid_files = glob(noncovid_path + '/*')

In [ ]:
print("First 5 Covid Files: ",covid_files[0:5])
print("Total Count: ",len(covid_files))
print("First 5 NonCovid Files: ",noncovid_files[0:5])
print("Total Count: ",len(noncovid_files))

First 5 Covid Files:  ['COVID/Covid (229).png', 'COVID/Covid (237).png', 'COVID/Covid (1251).png', 'COVID/Covid (182).png', 'COVID/Covid (262).png']
Total Count:  1252
First 5 NonCovid Files:  ['non-COVID/Non-Covid (1161).png', 'non-COVID/Non-Covid (1103).png', 'non-COVID/Non-Covid (1027).png', 'non-COVID/Non-Covid (1038).png', 'non-COVID/Non-Covid (123).png']
Total Count:  1229


In [ ]:
#lables generation
covid_labels = []
noncovid_labels = []
for i in range(len(covid_files)):
  covid_labels.append("covid")
for i in range(len(covid_files)):
  noncovid_labels.append("noncovid")

In [ ]:
files = covid_files + noncovid_files
lables = covid_labels + noncovid_labels
df = pd.DataFrame(list(zip(files, lables)),columns =['Files', 'Lables'])
df = df.sample(frac=1, random_state=42) #shuffle dataset

In [ ]:
label = []
featLength = 2+5+2
trainFeats = np.zeros((len(df),featLength)) #Feature vector of each image is of size 1x1030
for index in tqdm.tqdm_notebook(range(len(df))):
    img = PIL.Image.open(df["Files"][index]) #One image at a time
    img = img.resize((IMAGE_SIZE[0],IMAGE_SIZE[1]))
    img_gray = img.convert('L') #Converting to grayscale
    img_gray = PIL.ImageOps.equalize(img_gray, mask = None)
    img_arr = np.array(img_gray.getdata()).reshape(img.size[1],img.size[0]) #Converting to array
    # LBP
    feat_lbp = local_binary_pattern(img_arr,5,2,'uniform').reshape(img.size[0]*img.size[1])
    lbp_hist,_ = np.histogram(feat_lbp,8)
    lbp_hist = np.array(lbp_hist,dtype=float)
    lbp_prob = np.divide(lbp_hist,np.sum(lbp_hist))
    lbp_energy = np.nansum(lbp_prob**2)
    lbp_entropy = -np.nansum(np.multiply(lbp_prob,np.log2(lbp_prob)))   
    # GLCM
    gCoMat = greycomatrix(img_arr, [2], [0],256,symmetric=True, normed=True)
    contrast = greycoprops(gCoMat, prop='contrast')
    dissimilarity = greycoprops(gCoMat, prop='dissimilarity')
    homogeneity = greycoprops(gCoMat, prop='homogeneity')    
    energy = greycoprops(gCoMat, prop='energy')
    correlation = greycoprops(gCoMat, prop='correlation')    
    feat_glcm = np.array([contrast[0][0],dissimilarity[0][0],homogeneity[0][0],energy[0][0],correlation[0][0]])
    # Gabor filter
    gaborFilt_real,gaborFilt_imag = gabor(img_arr,frequency=0.6)
    gaborFilt = (gaborFilt_real**2+gaborFilt_imag**2)//2
    gabor_hist,_ = np.histogram(gaborFilt,8)
    gabor_hist = np.array(gabor_hist,dtype=float)
    gabor_prob = np.divide(gabor_hist,np.sum(gabor_hist))
    gabor_energy = np.nansum(gabor_prob**2)
    gabor_entropy = -np.nansum(np.multiply(gabor_prob,np.log2(gabor_prob)))
    # Concatenating features(2+5+2)    
    concat_feat = np.concatenate(([lbp_energy,lbp_entropy],feat_glcm,[gabor_energy,gabor_entropy]),axis=0)
    trainFeats[index,:] = concat_feat #Stacking features vectors for each image
    # Class label
    label.append(df["Lables"][index])
trainLabel = np.array(label) #Conversion from list to array

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


  0%|          | 0/2481 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: RuntimeWarning: divide by zero encountered in log2
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in multiply
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: RuntimeWarning: divide by zero encountered in log2
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: RuntimeWarning: invalid value encountered in multiply


In [ ]:
# Normalizing the train features to the range [0,1]
trMaxs = np.amax(trainFeats,axis=0) #Finding maximum along each column
trMins = np.amin(trainFeats,axis=0) #Finding maximum along each column
trMaxs_rep = np.tile(trMaxs,(2481,1)) #Repeating the maximum value along the rows
trMins_rep = np.tile(trMins,(2481,1)) #Repeating the minimum value along the rows
trainFeatsNorm = np.divide(trainFeats-trMins_rep,trMaxs_rep) #Element-wise division
# Normalizing the test features
tsMaxs_rep = np.tile(trMaxs,(10000,1)) #Repeating the maximum value along the rows
tsMins_rep = np.tile(trMins,(10000,1)) #Repeating the maximum value along the rows

NameError: ignored

In [ ]:
testFeatsNorm = np.divide(testFeats-tsMins_rep,tsMaxs_rep) #Element-wise division

In [ ]:
with open("trainFeatsNew.pckl", "wb") as f:
    pickle.dump(trainFeatsNorm, f)
with open("trainLabelNew.pckl", "wb") as f:
    pickle.dump(trainLabel, f)

In [ ]:

    
# Saving normalized testing data and labels
with open("testFeatsNew.pckl", "wb") as f:
    pickle.dump(testFeatsNorm, f)
with open("testLabelNew.pckl", "wb") as f:
    pickle.dump(testLabel, f)
    
print('Files saved to disk!')

NameError: ignored

In [86]:
X = pd.read_pickle('https://github.com/TharunNB/ML/blob/main/data/trainFeatsNew.pckl?raw=true')
Y = pd.read_pickle('https://github.com/M2kishore/Covid_Detection_using_CT_scans_ML/blob/M2kishore-dataset-with-file-lable/covidLable.pckl?raw=true')
X = pd.DataFrame(X,columns=['lbp_energy','lbp_entropy','glcm_contrast','glcm_dissimilarity','glcm_homogeniety','glcm_energy','glcm_correlation','gabor_energy','gabor_entropy'])
Y = pd.DataFrame(Y,columns=['labels'])

In [87]:
label_encoder = LabelEncoder()
Y['labels'] = label_encoder.fit_transform(Y["labels"])

In [88]:
X = X.sample(frac=1,random_state=42)
Y = Y.sample(frac=1,random_state=42)

In [89]:
X = StandardScaler().fit_transform(X)

#FEATURE DECOMPOSITION

In [112]:
X = pd.read_pickle('https://github.com/TharunNB/ML/blob/main/data/trainFeatsNew.pckl?raw=true')
Y = pd.read_pickle('https://github.com/M2kishore/Covid_Detection_using_CT_scans_ML/blob/M2kishore-dataset-with-file-lable/covidLable.pckl?raw=true')
X = pd.DataFrame(X,columns=['lbp_energy','lbp_entropy','glcm_contrast','glcm_dissimilarity','glcm_homogeniety','glcm_energy','glcm_correlation','gabor_energy','gabor_entropy'])
Y = pd.DataFrame(Y,columns=['labels'])
label_encoder = LabelEncoder()
Y['labels'] = label_encoder.fit_transform(Y["labels"])
X = X.sample(frac=1,random_state=42)
Y = Y.sample(frac=1,random_state=42)
Y = Y.values.reshape(-1,1)
X = StandardScaler().fit_transform(X)

#Function for Testing Random Forenst

In [113]:
def RandomForestTest(X, Y):
    X_Train, X_Test, Y_Train, Y_Test = train_test_split(X, Y, 
                                                        test_size = 0.30, 
                                                        random_state = 101)
    trainedforest = RandomForestClassifier(n_estimators=700).fit(X_Train,Y_Train)
    predictionforest = trainedforest.predict(X_Test)
    print(confusion_matrix(Y_Test,predictionforest))
    print(classification_report(Y_Test,predictionforest))

## PCA

In [114]:
def Pca(n):
  pca = PCA(n_components=n)
  x = pca.fit_transform(X)
  return x

In [115]:
X_pca = Pca(5)
RandomForestTest(X_pca,Y)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


[[341  57]
 [ 50 297]]
              precision    recall  f1-score   support

           0       0.87      0.86      0.86       398
           1       0.84      0.86      0.85       347

    accuracy                           0.86       745
   macro avg       0.86      0.86      0.86       745
weighted avg       0.86      0.86      0.86       745



## Fast ICA

In [116]:
def Ica(n):
  ica = FastICA(n_components=n)
  x = ica.fit_transform(X)
  return x

In [117]:
X_Ica = Ica(5)
RandomForestTest(Xdf,Y)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


[[341  57]
 [ 50 297]]
              precision    recall  f1-score   support

           0       0.87      0.86      0.86       398
           1       0.84      0.86      0.85       347

    accuracy                           0.86       745
   macro avg       0.86      0.86      0.86       745
weighted avg       0.86      0.86      0.86       745



## LDA

In [118]:
def Lda(n):
  lda = LinearDiscriminantAnalysis(n_components=n)

  # run an LDA and use it to transform the features
  x = lda.fit(X, Y).transform(X)
  print('Original number of features:', X.shape[1])
  print('Reduced number of features:', X_lda.shape[1])
  return x

In [122]:
X_lda = Lda(5)
RandomForestTest(X_lda,Y)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/discriminant_analysis.py:463: ChangedBehaviorWarning: n_components cannot be larger than min(n_features, n_classes - 1). Using min(n_features, n_classes - 1) = min(9, 2 - 1) = 1 components.
  ChangedBehaviorWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/discriminant_analysis.py:469: FutureWarning: In version 0.23, setting n_components > min(n_features, n_classes - 1) will raise a ValueError. You should set n_components to None (default), or a value smaller or equal to min(n_features, n_classes - 1).
  warnings.warn(future_msg, FutureWarning)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was

Original number of features: 9
Reduced number of features: 1
[[225 173]
 [140 207]]
              precision    recall  f1-score   support

           0       0.62      0.57      0.59       398
           1       0.54      0.60      0.57       347

    accuracy                           0.58       745
   macro avg       0.58      0.58      0.58       745
weighted avg       0.58      0.58      0.58       745



##LLE

In [124]:
def Lle(n):
  embedding = LocallyLinearEmbedding(n_components=3)
  x = embedding.fit_transform(X)
  return x

In [125]:
X_lle = Lle(5)
RandomForestTest(X_lle,Y)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


[[232 166]
 [110 237]]
              precision    recall  f1-score   support

           0       0.68      0.58      0.63       398
           1       0.59      0.68      0.63       347

    accuracy                           0.63       745
   macro avg       0.63      0.63      0.63       745
weighted avg       0.64      0.63      0.63       745



## t-SNE

In [128]:
def Tsne(n,verbose,preplexity,n_iter):
  tsne = TSNE(n_components=3, verbose=1, perplexity=40, n_iter=300)
  x = tsne.fit_transform(X)
  return x

In [129]:
X_tsne = Tsne(5,1,40,3000)
RandomForestTest(X_lle,Y)

[t-SNE] Computing 121 nearest neighbors...
[t-SNE] Indexed 2481 samples in 0.002s...
[t-SNE] Computed neighbors for 2481 samples in 0.087s...
[t-SNE] Computed conditional probabilities for sample 1000 / 2481
[t-SNE] Computed conditional probabilities for sample 2000 / 2481
[t-SNE] Computed conditional probabilities for sample 2481 / 2481
[t-SNE] Mean sigma: 0.391852
[t-SNE] KL divergence after 250 iterations with early exaggeration: 63.944981
[t-SNE] KL divergence after 300 iterations: 0.806496


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


[[234 164]
 [108 239]]
              precision    recall  f1-score   support

           0       0.68      0.59      0.63       398
           1       0.59      0.69      0.64       347

    accuracy                           0.63       745
   macro avg       0.64      0.64      0.63       745
weighted avg       0.64      0.63      0.63       745



## Autoencoders

In [134]:
from keras.layers import Input, Dense
from keras.models import Model

input_layer = Input(shape=(X.shape[1],))
encoded = Dense(7, activation='relu')(input_layer)
decoded = Dense(X.shape[1], activation='softmax')(encoded)
autoencoder = Model(input_layer, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

X1, X2, Y1, Y2 = train_test_split(X, X, test_size=0.3, random_state=101)

autoencoder.fit(X1, Y1,
                epochs=100,
                batch_size=300,
                shuffle=True,
                verbose = 30,
                validation_data=(X2, Y2))

encoder = Model(input_layer, encoded)
X_ae = encoder.predict(X)

Epoch 1/100
Epoch 2/100
Epoch 3/100
Epoch 4/100
Epoch 5/100
Epoch 6/100
Epoch 7/100
Epoch 8/100
Epoch 9/100
Epoch 10/100
Epoch 11/100
Epoch 12/100
Epoch 13/100
Epoch 14/100
Epoch 15/100
Epoch 16/100
Epoch 17/100
Epoch 18/100
Epoch 19/100
Epoch 20/100
Epoch 21/100
Epoch 22/100
Epoch 23/100
Epoch 24/100
Epoch 25/100
Epoch 26/100
Epoch 27/100
Epoch 28/100
Epoch 29/100
Epoch 30/100
Epoch 31/100
Epoch 32/100
Epoch 33/100
Epoch 34/100
Epoch 35/100
Epoch 36/100
Epoch 37/100
Epoch 38/100
Epoch 39/100
Epoch 40/100
Epoch 41/100
Epoch 42/100
Epoch 43/100
Epoch 44/100
Epoch 45/100
Epoch 46/100
Epoch 47/100
Epoch 48/100
Epoch 49/100
Epoch 50/100
Epoch 51/100
Epoch 52/100
Epoch 53/100
Epoch 54/100
Epoch 55/100
Epoch 56/100
Epoch 57/100
Epoch 58/100
Epoch 59/100
Epoch 60/100
Epoch 61/100
Epoch 62/100
Epoch 63/100
Epoch 64/100
Epoch 65/100
Epoch 66/100
Epoch 67/100
Epoch 68/100
Epoch 69/100
Epoch 70/100
Epoch 71/100
Epoch 72/100
Epoch 73/100
Epoch 74/100
Epoch 75/100
Epoch 76/100
Epoch 77/100
Epoch 78

In [135]:
RandomForestTest(X_ae,Y)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


[[305  93]
 [ 77 270]]
              precision    recall  f1-score   support

           0       0.80      0.77      0.78       398
           1       0.74      0.78      0.76       347

    accuracy                           0.77       745
   macro avg       0.77      0.77      0.77       745
weighted avg       0.77      0.77      0.77       745

